# 12.1 Categorical Data

## Background and Motivation

In [1]:
import numpy as np
import pandas as pd

values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [2]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [3]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

### `take`
Apply on the dimension table, take argument the integer series you want to restore.


In data warehousing, a best practice is to use so called *dimension tables* containing the distinct values and storing the primary observations as integer keys referencing the dimension table.

In [6]:
values2 = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values2

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
dim.take(values2)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

- The array of distinct values (`dim`) can be called <mark>*categories*</mark>.
- This representation (index of `dim`) as integers is called the <mark>*categorical* representation</mark>.
- The integer values (`values2`) that reference the categories are called *category codes* or simply <mark>*codes*</mark>.

## Categorical Type in pandas
pandas has a special `Categorical` type for holding data that uses the integer-based categorical representation or *encoding*.

In [13]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.randint(0, 4, size=N),},
                    columns=['basket_id', 'fruit', 'count', 'weight']
                   )
df

,basket_id,fruit,count,weight
0,0,apple,10,0
1,1,orange,10,1
2,2,apple,3,0
3,3,apple,12,2
4,4,apple,9,3
5,5,orange,6,2
6,6,apple,6,0
7,7,apple,5,0


### `astype`
Here `df['fruit']` is an array of Python string objects. We will convert it to categorical.

In [14]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

The values for `fruit_cat` are an instance of `pandas.Categorical`.

In [15]:
c = fruit_cat.values
c

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
Categories (2, object): ['apple', 'orange']

In [17]:
type(c)

pandas.core.arrays.categorical.Categorical

The `Categorical` object has `categories` and `codes` attributes.

In [18]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [19]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [20]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

You can also create `pandas.Categorical` directly from other types of Python sequences

In [21]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

### `pandas.Categorical.from_codes(codes, categories)`
If you have obtained categorical encoded data from another source.

In [22]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)

my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

Unless explicitly specified, categorical conversions assume no specific ordering of the categories. You can indicate that the catgories have a meaningful ordering.

In [23]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

An ordered categorical instance can be made ordered with `as_ordered`.

In [24]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

## Computations with Categoricals
`pandas.qcut` binning function returns `pandas.Categorical`

In [25]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [26]:
# Let's compute a quantile binning of this data and extract some information
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

Passing `labels` argument let us give quartile names.

In [27]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [29]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

The labeled `bins` categorical does not contain information about the bin edges in the data, So we can use `groupby` to extract some summary statistics.

In [31]:
bins = pd.Series(bins, name='quartile')

results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


The `quartile` column in the result retains original categorical information, including ordering, from `bins`.

In [33]:
results.quartile

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

### Better performance with categoricals

In [34]:
N = 100_00_000
draws = pd.Series(np.random.randn(N))

# (4 * N/4 ) == N
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N//4))

Now we convert `labels` to categorical.

In [36]:
categories = labels.astype('category')

Now we note that `labels` uses significantly more memory than `categories`.

In [37]:
labels.memory_usage()

80000128

In [38]:
categories.memory_usage()

10000320

## Categorical Methods
Series containing categorical data have special methods similar to the `Series.str` specialized string methods.

In [39]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

The special attribute `cat` provides access to categorical methods.

In [40]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [41]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

### `set_categories`
#### Change/Update the categories for the series.

In [42]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [43]:
# The new categories will be affected in operations that use them
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [44]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

After you filter a large DataFrame or Series, many of the categories may not appear in the data. To help with this use the `remove_unused_categories` method to trim unobserved categories.

In [45]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [47]:
cat_s3.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [48]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

### Creating dummy categories for modeling
### `pandas.get_dummies`
Creating *dummy variables* also known as *one-hot* encoding; which involves creating a DataFrame with a Column for each distinct category. These columns contain 1s for occurences of a given category and 0 otherwise.

In [49]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


# 12.2 Advanced GroupBy Use
## Group Transforms and "Unwrapped" GroupBys
After looking at `apply` we will look at another similar method `transform`; but imposes more contraint on the kind f function you can use:

- It can produce a <mark>scalar value</mark> to be broadcast to the shape of the group
- It can produce an <mark>object of the same shape as the input group</mark>.
- It <mark>must not mutate its input</mark>.

In [50]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 
                  'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [56]:
# Here are group means by key
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

Suppose we wanted to produce a Series of the same shape as `df['value']` but with values replaces by the average grouped by `'key'`. We can pass the function `lambda x: x.mean()` to `transform`.

In [59]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

For built-in aggregation functions, we can pass a string alias as with the GroupBy `agg` method:

In [60]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [63]:
g.apply(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [64]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [65]:
# We can compute the ranks in descending order for each group:
g.transform(lambda x: x.rank(ascending=False)) 

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [66]:
# Consider a group transformation function composed from simple aggregations:
def normalize(x):
    return (x - x.mean()) / x.std()

# We can obtain equivalent results in this case using transform or apply
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [67]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

Built-in aggregate functions like `'mean'` or `'sum'` are often much faster than a general `apply` functions. These also have a "fast part" when used with `transform`. This allows us to perform a so-called *unwrapped group* operation

In [69]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [70]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

While an unwrapped group operations may involve multiple group aggregations, the overall benefit of vectorized operations often outweighs this.

## Grouped Time Resampling

In [72]:
N = 15
times = pd.date_range('2021-01-20 00:00', freq='1T', periods=N)
df = pd.DataFrame({'time': times, 
                   'values': np.arange(N)})
df

,time,values
0,2021-01-20 00:00:00,0
1,2021-01-20 00:01:00,1
2,2021-01-20 00:02:00,2
3,2021-01-20 00:03:00,3
4,2021-01-20 00:04:00,4
5,2021-01-20 00:05:00,5
6,2021-01-20 00:06:00,6
7,2021-01-20 00:07:00,7
8,2021-01-20 00:08:00,8
9,2021-01-20 00:09:00,9


Here we can index by `time` and then `resample`.

In [74]:
df.set_index('time').resample('5T', label='right').count()

,values
time,
2021-01-20 00:05:00,5
2021-01-20 00:10:00,5
2021-01-20 00:15:00,5


Suppose that a **DataFrame contains multiple time series**, marked by an additional group key column.

In [95]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2

,time,key,value
0,2021-01-20 00:00:00,a,0.0
1,2021-01-20 00:00:00,b,1.0
2,2021-01-20 00:00:00,c,2.0
3,2021-01-20 00:01:00,a,3.0
4,2021-01-20 00:01:00,b,4.0
5,2021-01-20 00:01:00,c,5.0
6,2021-01-20 00:02:00,a,6.0
7,2021-01-20 00:02:00,b,7.0
8,2021-01-20 00:02:00,c,8.0
9,2021-01-20 00:03:00,a,9.0


### `pandas.Grouper`
To do the same resampling for each value of '`key`'

In [96]:
time_key = pd.Grouper(freq='5min')

We can set the time index, group by '`key`' and `time_key`, and aggregate:

In [97]:
resampled = (df2.set_index('time')
             .groupby(['key', time_key])
             .sum())
resampled

value
key time                      
a   2021-01-20 00:00:00   30.0
    2021-01-20 00:05:00  105.0
    2021-01-20 00:10:00  180.0
b   2021-01-20 00:00:00   35.0
    2021-01-20 00:05:00  110.0
    2021-01-20 00:10:00  185.0
c   2021-01-20 00:00:00   40.0
    2021-01-20 00:05:00  115.0
    2021-01-20 00:10:00  190.0

In [89]:
resampled.reset_index()

,key,time,value
0,a,2021-01-20 00:00:00,30.0
1,a,2021-01-20 00:05:00,105.0
2,a,2021-01-20 00:10:00,180.0
3,b,2021-01-20 00:00:00,35.0
4,b,2021-01-20 00:05:00,110.0
5,b,2021-01-20 00:10:00,185.0
6,c,2021-01-20 00:00:00,40.0
7,c,2021-01-20 00:05:00,115.0
8,c,2021-01-20 00:10:00,190.0


# 12.3 Techniques for Method Chaining
### `pandas.DataFrame.assign`
This method is a *functional* alternative to column assignments of the form `df[k] = v`. Rather than modifying the object in-place, it return a new DataFrame with the indicated modification. So these are equivalent.

In [ ]:
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)

Assigning in-place may execute faster than using `assign`, but `assign` enables easier method chaining:

In [ ]:
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())

We cannot refer to the result of `load_data` until is has been assigned to the temporary varaible `df`. To help with this, `assign` and many other pandas function accept function-like arguments, also known as *callables*.

In [ ]:
df = load_data()
df2 = df[ df['col2']<0 ]

This can be rewritten as:

In [ ]:
df = (load_data()[lambda x: x[ 'col2']<0 ])

<mark>Here, the result of `load_data` is not assigned to a variable, so the function passed into `[]` is then *bound* to the object at that stage of the method chain.</mark>

We can continue , then, and write the entire sequence as a single chained expression:

In [ ]:
result = (load_data()
           [ lambda x: x.col2 < 0 ]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())

## The pipe Method
The pipe method can help you use multiple different functions to sort-of chained one; passing the output as input to another. Consider a sequence of function calls:

In [ ]:
a = f(df, arg1=v1)
b = g(a, arg2=v3)
c = h(b, arg3=v4)

When using functions that accept and return Series or DataFrame object, you can rewrite it as:

In [ ]:
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg3=v4))

A potentially useful pattern for `pipe` is to generalize sequences of operations into reusable functions.

In [ ]:
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')

Suppose you wanted to be able to demean more than one column and easily change the group keys. Additionally, you might want to perform this transformation in a mathod chain. Here is how you do it,

In [ ]:
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c]= df[c] - g[c].transform('mean')
    return result

result = (df[ df.col1<0 ]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))